In [4]:
import json
import os
import pandas as pd
import requests
from tqdm import tqdm
import psycopg2


In [63]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()


In [20]:
word = []
#definition = []
local_word = []
examples = []
examples_local = []
parent = "./y_lookup/en_ru/"
for file in os.listdir(parent):
    with open(os.path.join(parent,file), "r") as f: 
        data = json.load(f)
        for definition in data['def']:
            word_current =  definition['text']
            for translation in definition['tr']: 
                word.append(word_current)
                local_word.append(translation['text'])
                ex_en = []
                ex_rus = []
                if 'ex' in translation:
                    for exmpl in translation['ex']:
                        #print(exmpl)
                        ex_en.append(exmpl['text'])
                        ex_rus.append(exmpl['tr'][0]['text'])
                examples.append(ex_en)
                examples_local.append(ex_rus)
                #print(translation)
                #print()
    

In [22]:
def parse_from_yandex_lookup(directory, print_output = False):
    word = []
    local_word = []
    examples = []
    examples_local = []
    
    for file in os.listdir(directory):
        if print_output == True: print(file)
        with open(os.path.join(directory,file), "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    data = pd.DataFrame(list(zip(word,local_word,examples, examples_local)),columns =['word', 'local_word', 'examples','local_examples'])
    return  data


In [69]:
def get_word_forms(word):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        print(word, " is not in db")
        return []
    else:
        #print(a[0])
        word_id = req_res[0]
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    word_forms = []
    for word in cursor:
        #print(word[0])
        word_forms.append(word[0])
    return word_forms
get_word_forms("book")


['booker', 'booked', 'booking', 'books']

In [71]:
def parse_json(path, word, local_word,examples,examples_local):
    try:
        with open(path, "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    except:
        pass

In [72]:
def parse_from_yandex_lookup_vs_wordforms(directory):
    word = []
    local_word = []
    examples = []
    examples_local = []
    words = pd.read_csv("test_words.csv", header = None)
    for word_i in words[0].items():
        print (word_i[1])
        file = word_i[1] + ".json"
        parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        wordforms = get_word_forms(word_i[1].lower())
        for wf in wordforms:
            print(">>>", wf)
            file = wf + ".json"
            parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        
    data = pd.DataFrame(list(zip(word,local_word,examples, examples_local)),columns =['word', 'local_word', 'examples','local_examples'])
    return  data
enru_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_ru/")

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

>>> befores
begin
>>> begins
>>> began
>>> beginning
>>> beginer
>>> begining
>>> beginnest
>>> begun
>>> begin
behind
>>> behinds
below
>>> below
>>> belowed
>>> belows
best
>>> bester
>>> bests
>>> best best
>>> besting
>>> bestest
>>> bested
synchrophasotron
synchrophasotron  is not in db


In [75]:
enru_wv.to_csv("yandex_lookup_en_ru_with_wordforms.csv")

In [74]:
enru_wv.head(30)

,word,local_word,examples,local_examples
0,word,слово,"[last word, old english word, new word in art,...","[последнее слово, старое английское слово, нов..."
1,word,известие,[],[]
2,word,слова,[],[]
3,word,говорить,[],[]
4,word,сформулировать,[],[]
5,word,текстовый,[word processing document],[текстовый документ]
6,word,словесный,[word game],[словесная игра]
7,wording,формулировка,[exact wording],[точная формулировка]
8,wording,редакция,[new wording],[новая редакция]
9,wording,текст,[wording of the declaration],[текст декларации]


In [76]:
en_fr_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_fr/")

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

>>> befores
begin
>>> begins
>>> began
>>> beginning
>>> beginer
>>> begining
>>> beginnest
>>> begun
>>> begin
behind
>>> behinds
below
>>> below
>>> belowed
>>> belows
best
>>> bester
>>> bests
>>> best best
>>> besting
>>> bestest
>>> bested
synchrophasotron
synchrophasotron  is not in db


In [78]:
en_fr_wv.to_csv("yandex_lookup_en_fr_with_wordforms.csv")

In [21]:
len(word), len(local_word), len(examples), len(examples_local)

(676, 676, 676, 676)

In [35]:
enru_ = parse_from_yandex_lookup("./y_lookup/en_ru/")
enru.head()

,word,local_word,examples,local_examples
0,for,для,"[for the bank, for last month, for one year, f...","[для банка, за последний месяц, на один год, н..."
1,for,ради,[for god],[ради бога]
2,for,в течение,[],[]
3,for,в отношении,[],[]
4,for,для целей,[],[]


In [ ]:
enru

In [23]:
enfr__whole_list = parse_from_yandex_lookup("./y_lookup/en_fr_whole_list/", print_output=True)
enfr__whole_list.head()

for.json
before.json
adverb.json
during.json
airport.json
exit.json
drum.json
empty.json
dressed.json
cafe.json
flight.json
begin.json
fact.json
dream.json
alarm.json
along.json
follow.json
warm.json
collide.json
excellent.json
break.json
examination.json
tiger.json
engineer.json
lion.json
even.json
field.json
background.json
easily.json
write.json
ravioli.json
apologise.json
fire.json
fly.json
beaver.json
duck.json
coincide.json
exactly.json
anything.json
exhaustively.json
apoplexy.json
coconut.json
not.json
international.json
synchrophasotron.json
best.json
parent.json
ocean.json
coincidental.json
buzzer.json
century.json
euro.json
dynamite.json
buy.json
afraid.json
peremptory.json
member.json
childhood.json
go.json
plate.json
fish.json
farmer.json
stand.json
drawing.json
explain.json
typhus.json
precedent.json
advertisement.json
issue.json
fifty.json
thousand.json
aged.json
file.json
pen.json
fan.json
pasta.json
hot.json
that.json
engine.json
advanced.json
be.json
but.json
and.json


,word,local_word,examples,local_examples
0,for,pour,[],[]
1,before,avant,[],[]
2,before,auparavant,[],[]
3,adverb,adverbe,[],[]
4,during,lors,[],[]


In [27]:
list(enfr__whole_list['word']).index("nobody")

ValueError: 'nobody' is not in list

In [39]:
enru.to_csv("yandex_lookup_en_ru_lemmas_only.csv")

In [21]:
enfr__whole_list.to_csv("yandex_lookup_en_fr_lemmas_only_WHOLE_LIST.csv")

In [44]:
url_y = "http://192.168.122.13:31436/GetTranslate"
h_y = {"apiVersion": "1.0.0", "text": "гайки и болты", "langPair": {"source": "ru", "target": "fr"}}
#h_y = {"apiVersion": "1.0.0", "text": "Landing to the mars", "langPair": "en-ru"}
r = requests.post(url = url_y, json=h_y)
data = r.json()
data

{'translate': 'écrous et boulons'}

In [29]:
def y_translate(text, from_lang, to_lang):
    url_y = "http://192.168.122.13:31436/GetTranslate"
    h_y = {"apiVersion": "1.0.0", "text": text, "langPair": {"source": from_lang, "target": to_lang}}
    r = requests.post(url = url_y, json=h_y)
    data = r.json()
    return data['translate']
y_translate("привет уеба", "ru", "fr")

'salut, connard.'

In [14]:
enru_small = parse_from_yandex_lookup("./y_lookup/en_ru/")
len(enru_small)

1190

19

1410

In [15]:
enru_big = parse_from_yandex_lookup("./y_lookup/yandex_lookup_big_en_ru/")
print(len(enru_big))
enru_big.head()

1410


,word,local_word,examples,local_examples
0,for,для,"[for the bank, for last month, for one year, f...","[для банка, за последний месяц, на один год, н..."
1,for,ради,[for god],[ради бога]
2,for,в течение,[],[]
3,for,в отношении,[],[]
4,for,для целей,[],[]


In [33]:
def naive_translation(en_ru_dataframe, from_lang,to_lang):
    word = []
    local_word = []
    example = []
    local_example = []
    for element_ind in tqdm(range(len(en_ru_dataframe))):
        el_list = list(en_ru_dataframe.iloc[element_ind])
        #print(el_list)
        word.append(el_list[0])
        try:
            tr_local_word = y_translate(el_list[1],from_lang,to_lang)
        except:
            tr_local_word = "no_translation"
            print("failed", el_list[1])
        local_word.append(tr_local_word)
        ex = []
        loc_ex = []
        for example_i in el_list[2]:
            ex.append(example_i)
            try:
                ex_tr = y_translate(example_i,"en",to_lang)
            except:
                ex_tr = "no_translation"
                print("failed", example_i)
            loc_ex.append(ex_tr)
        example.append(ex)
        local_example.append(loc_ex)
    
    data = pd.DataFrame(list(zip(word,local_word,example, local_example)),columns =['word', 'local_word', 'examples','local_examples'])
    return data

#df_enru_whole = naive_translation(enru_big,"ru","fr")
df_enru_whole.head()

,word,local_word,examples,local_examples
0,for,pour,"[for the bank, for last month, for one year, f...","[pour la banque, pour le mois dernier, pendant..."
1,for,pour,[for god],[pour dieu]
2,for,au cours de,[],[]
3,for,à l'égard de,[],[]
4,for,à des fins,[],[]


In [18]:
df_enru_whole.to_csv("naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")

# SYSTRAN EN RU TO FR NAIVE

In [28]:
df_systran_en_ru = pd.read_csv("systran_en_ru.csv")

In [32]:
df_systran_en_ru_fr = naive_translation(df_systran_en_ru,"ru","fr")



  0%|          | 0/1358 [00:00<?, ?it/s]

  0%|          | 1/1358 [00:03<1:09:22,  3.07s/it]

  0%|          | 2/1358 [00:04<56:51,  2.52s/it]  

  0%|          | 3/1358 [00:05<46:25,  2.06s/it]

  0%|          | 4/1358 [00:05<36:36,  1.62s/it]

  0%|          | 5/1358 [00:07<37:03,  1.64s/it]

  0%|          | 6/1358 [00:08<30:50,  1.37s/it]

  1%|          | 7/1358 [00:11<44:55,  1.99s/it]

  1%|          | 8/1358 [00:12<37:34,  1.67s/it]

  1%|          | 9/1358 [00:24<1:45:07,  4.68s/it]

  1%|          | 10/1358 [00:25<1:24:19,  3.75s/it]

  1%|          | 11/1358 [00:27<1:07:06,  2.99s/it]

  1%|          | 12/1358 [00:29<1:02:21,  2.78s/it]

  1%|          | 13/1358 [00:31<54:12,  2.42s/it]  

  1%|          | 14/1358 [00:32<45:33,  2.03s/it]

  1%|          | 15/1358 [00:33<38:33,  1.72s/it]

  1%|          | 16/1358 [00:33<31:59,  1.43s/it]

  1%|▏         | 17/1358 [00:35<34:50,  1.56s/it]

  1%|▏         | 18/1358 [00:36<29:42,  1.33s/it]

  1%|▏         | 19/1358 [00:44<1

 10%|█         | 139/1358 [09:10<1:13:05,  3.60s/it]

 10%|█         | 140/1358 [09:15<1:21:43,  4.03s/it]

 10%|█         | 141/1358 [09:17<1:09:21,  3.42s/it]

 10%|█         | 142/1358 [09:23<1:23:27,  4.12s/it]

 11%|█         | 143/1358 [09:27<1:22:04,  4.05s/it]

 11%|█         | 144/1358 [09:30<1:19:17,  3.92s/it]

 11%|█         | 145/1358 [09:32<1:03:44,  3.15s/it]

 11%|█         | 146/1358 [09:35<1:02:57,  3.12s/it]

 11%|█         | 147/1358 [09:37<57:26,  2.85s/it]  

 11%|█         | 148/1358 [09:39<51:28,  2.55s/it]

 11%|█         | 149/1358 [09:41<52:16,  2.59s/it]

 11%|█         | 150/1358 [09:45<56:21,  2.80s/it]

 11%|█         | 151/1358 [09:48<1:01:25,  3.05s/it]

 11%|█         | 152/1358 [09:51<59:58,  2.98s/it]  

 11%|█▏        | 153/1358 [09:53<51:00,  2.54s/it]

 11%|█▏        | 154/1358 [09:56<55:17,  2.76s/it]

 11%|█▏        | 155/1358 [10:01<1:11:28,  3.57s/it]

 11%|█▏        | 156/1358 [10:05<1:11:17,  3.56s/it]

 12%|█▏        | 157/1358 [10:10<1:22:

 20%|██        | 275/1358 [18:48<1:24:33,  4.68s/it]

 20%|██        | 276/1358 [18:52<1:20:44,  4.48s/it]

 20%|██        | 277/1358 [18:56<1:16:02,  4.22s/it]

 20%|██        | 278/1358 [19:01<1:19:15,  4.40s/it]

 21%|██        | 279/1358 [19:04<1:10:35,  3.92s/it]

 21%|██        | 280/1358 [19:09<1:16:09,  4.24s/it]

 21%|██        | 281/1358 [19:11<1:05:12,  3.63s/it]

 21%|██        | 282/1358 [19:14<1:03:39,  3.55s/it]

 21%|██        | 283/1358 [19:16<53:05,  2.96s/it]  

 21%|██        | 284/1358 [19:18<48:03,  2.68s/it]

 21%|██        | 285/1358 [19:21<49:09,  2.75s/it]

 21%|██        | 286/1358 [19:24<50:36,  2.83s/it]

 21%|██        | 287/1358 [19:25<42:51,  2.40s/it]

 21%|██        | 288/1358 [19:29<52:28,  2.94s/it]

 21%|██▏       | 289/1358 [19:32<51:14,  2.88s/it]

 21%|██▏       | 290/1358 [19:37<1:02:40,  3.52s/it]

 21%|██▏       | 291/1358 [19:42<1:07:28,  3.79s/it]

 22%|██▏       | 292/1358 [19:44<1:00:01,  3.38s/it]

 22%|██▏       | 293/1358 [19:48<1:02:53

 30%|███       | 411/1358 [30:03<1:25:21,  5.41s/it]

 30%|███       | 412/1358 [32:07<10:48:37, 41.14s/it]

 30%|███       | 413/1358 [32:09<7:42:33, 29.37s/it] 

 30%|███       | 414/1358 [32:12<5:37:47, 21.47s/it]

 31%|███       | 415/1358 [32:15<4:08:39, 15.82s/it]

 31%|███       | 416/1358 [32:19<3:12:40, 12.27s/it]

 31%|███       | 417/1358 [32:25<2:45:58, 10.58s/it]

 31%|███       | 418/1358 [32:29<2:15:18,  8.64s/it]

 31%|███       | 419/1358 [32:33<1:49:07,  6.97s/it]

 31%|███       | 420/1358 [32:34<1:24:39,  5.42s/it]

 31%|███       | 421/1358 [32:38<1:16:17,  4.88s/it]

 31%|███       | 422/1358 [32:42<1:14:14,  4.76s/it]

 31%|███       | 423/1358 [32:45<1:04:32,  4.14s/it]

 31%|███       | 424/1358 [32:48<58:45,  3.77s/it]  

 31%|███▏      | 425/1358 [32:50<51:49,  3.33s/it]

 31%|███▏      | 426/1358 [32:52<45:41,  2.94s/it]

 31%|███▏      | 427/1358 [32:55<42:09,  2.72s/it]

 32%|███▏      | 428/1358 [32:59<49:46,  3.21s/it]

 32%|███▏      | 429/1358 [33:05<1

 40%|████      | 549/1358 [43:00<1:19:58,  5.93s/it]

 41%|████      | 550/1358 [43:02<1:03:55,  4.75s/it]

 41%|████      | 551/1358 [43:05<55:46,  4.15s/it]  

 41%|████      | 552/1358 [43:09<54:58,  4.09s/it]

 41%|████      | 553/1358 [43:15<1:01:50,  4.61s/it]

 41%|████      | 554/1358 [43:18<55:18,  4.13s/it]  

 41%|████      | 555/1358 [43:21<52:21,  3.91s/it]

 41%|████      | 556/1358 [43:25<52:19,  3.91s/it]

 41%|████      | 557/1358 [43:28<50:21,  3.77s/it]

 41%|████      | 558/1358 [43:34<57:55,  4.34s/it]

 41%|████      | 559/1358 [43:39<59:31,  4.47s/it]

 41%|████      | 560/1358 [43:42<52:25,  3.94s/it]

 41%|████▏     | 561/1358 [43:44<47:05,  3.55s/it]

 41%|████▏     | 562/1358 [43:46<41:34,  3.13s/it]

 41%|████▏     | 563/1358 [43:49<39:52,  3.01s/it]

 42%|████▏     | 564/1358 [43:53<42:23,  3.20s/it]

 42%|████▏     | 565/1358 [43:59<53:22,  4.04s/it]

 42%|████▏     | 566/1358 [44:05<1:01:31,  4.66s/it]

 42%|████▏     | 567/1358 [44:11<1:07:33,  5.12s/it]

 51%|█████     | 688/1358 [52:56<30:52,  2.76s/it]

 51%|█████     | 689/1358 [52:58<30:31,  2.74s/it]

 51%|█████     | 690/1358 [53:00<27:20,  2.46s/it]

 51%|█████     | 691/1358 [53:03<28:57,  2.60s/it]

 51%|█████     | 692/1358 [53:06<30:20,  2.73s/it]

 51%|█████     | 693/1358 [53:11<38:40,  3.49s/it]

 51%|█████     | 694/1358 [53:16<43:23,  3.92s/it]

 51%|█████     | 695/1358 [53:18<37:18,  3.38s/it]

 51%|█████▏    | 696/1358 [53:24<44:01,  3.99s/it]

 51%|█████▏    | 697/1358 [53:26<36:29,  3.31s/it]

 51%|█████▏    | 698/1358 [53:30<40:17,  3.66s/it]

 51%|█████▏    | 699/1358 [53:33<38:47,  3.53s/it]

 52%|█████▏    | 700/1358 [53:36<37:27,  3.42s/it]

 52%|█████▏    | 701/1358 [53:39<35:04,  3.20s/it]

 52%|█████▏    | 702/1358 [53:42<33:41,  3.08s/it]

 52%|█████▏    | 703/1358 [53:43<28:37,  2.62s/it]

 52%|█████▏    | 704/1358 [53:47<32:41,  3.00s/it]

 52%|█████▏    | 705/1358 [53:50<31:32,  2.90s/it]

 52%|█████▏    | 706/1358 [53:52<29:49,  2.74s/it]

 52%|█████▏ 

 61%|██████    | 827/1358 [1:02:47<39:03,  4.41s/it]

 61%|██████    | 828/1358 [1:02:50<34:37,  3.92s/it]

 61%|██████    | 829/1358 [1:02:54<35:20,  4.01s/it]

 61%|██████    | 830/1358 [1:02:58<34:46,  3.95s/it]

 61%|██████    | 831/1358 [1:03:04<38:48,  4.42s/it]

 61%|██████▏   | 832/1358 [1:03:05<31:53,  3.64s/it]

 61%|██████▏   | 833/1358 [1:03:08<29:32,  3.38s/it]

 61%|██████▏   | 834/1358 [1:03:12<30:32,  3.50s/it]

 61%|██████▏   | 835/1358 [1:03:18<37:36,  4.31s/it]

 62%|██████▏   | 836/1358 [1:03:20<31:45,  3.65s/it]

 62%|██████▏   | 837/1358 [1:03:24<32:04,  3.69s/it]

 62%|██████▏   | 838/1358 [1:03:28<32:35,  3.76s/it]

 62%|██████▏   | 839/1358 [1:03:33<36:30,  4.22s/it]

 62%|██████▏   | 840/1358 [1:03:39<40:51,  4.73s/it]

 62%|██████▏   | 841/1358 [1:03:44<40:42,  4.72s/it]

 62%|██████▏   | 842/1358 [1:03:47<36:15,  4.22s/it]

 62%|██████▏   | 843/1358 [1:03:50<34:12,  3.99s/it]

 62%|██████▏   | 844/1358 [1:03:54<32:58,  3.85s/it]

 62%|██████▏   | 845/1358 [1

 71%|███████   | 963/1358 [1:13:41<32:40,  4.96s/it]

 71%|███████   | 964/1358 [1:13:48<35:17,  5.37s/it]

 71%|███████   | 965/1358 [1:13:56<41:06,  6.28s/it]

 71%|███████   | 966/1358 [1:14:00<36:13,  5.55s/it]

 71%|███████   | 967/1358 [1:14:04<32:48,  5.04s/it]

 71%|███████▏  | 968/1358 [1:14:09<32:39,  5.02s/it]

 71%|███████▏  | 969/1358 [1:14:17<38:52,  6.00s/it]

 71%|███████▏  | 970/1358 [1:14:20<33:16,  5.14s/it]

 72%|███████▏  | 971/1358 [1:14:23<29:44,  4.61s/it]

 72%|███████▏  | 972/1358 [1:14:27<28:30,  4.43s/it]

 72%|███████▏  | 973/1358 [1:14:30<24:17,  3.79s/it]

 72%|███████▏  | 974/1358 [1:14:34<25:54,  4.05s/it]

 72%|███████▏  | 975/1358 [1:14:39<26:49,  4.20s/it]

 72%|███████▏  | 976/1358 [1:14:42<25:17,  3.97s/it]

 72%|███████▏  | 977/1358 [1:14:46<25:16,  3.98s/it]

 72%|███████▏  | 978/1358 [1:14:52<29:02,  4.59s/it]

 72%|███████▏  | 979/1358 [1:14:58<30:19,  4.80s/it]

 72%|███████▏  | 980/1358 [1:15:04<33:41,  5.35s/it]

 72%|███████▏  | 981/1358 [1

 81%|████████  | 1097/1358 [1:25:17<23:02,  5.30s/it]

 81%|████████  | 1098/1358 [1:25:24<24:33,  5.67s/it]

 81%|████████  | 1099/1358 [1:25:29<23:52,  5.53s/it]

 81%|████████  | 1100/1358 [1:25:34<23:24,  5.44s/it]

 81%|████████  | 1101/1358 [1:25:44<28:16,  6.60s/it]

 81%|████████  | 1102/1358 [1:25:49<26:00,  6.09s/it]

 81%|████████  | 1103/1358 [1:25:53<23:26,  5.52s/it]

 81%|████████▏ | 1104/1358 [1:26:02<27:37,  6.53s/it]

 81%|████████▏ | 1105/1358 [1:26:08<27:06,  6.43s/it]

 81%|████████▏ | 1106/1358 [1:26:14<26:36,  6.34s/it]

 82%|████████▏ | 1107/1358 [1:26:24<30:49,  7.37s/it]

 82%|████████▏ | 1108/1358 [1:26:32<32:24,  7.78s/it]

 82%|████████▏ | 1109/1358 [1:26:37<28:38,  6.90s/it]

 82%|████████▏ | 1110/1358 [1:26:39<22:06,  5.35s/it]

 82%|████████▏ | 1111/1358 [1:26:43<20:04,  4.88s/it]

 82%|████████▏ | 1112/1358 [1:26:46<17:40,  4.31s/it]

 82%|████████▏ | 1113/1358 [1:26:53<21:09,  5.18s/it]

 82%|████████▏ | 1114/1358 [1:27:02<25:55,  6.38s/it]

 82%|█████

 91%|█████████ | 1231/1358 [1:35:45<09:51,  4.65s/it]

 91%|█████████ | 1232/1358 [1:35:49<09:31,  4.54s/it]

 91%|█████████ | 1233/1358 [1:35:52<08:48,  4.23s/it]

 91%|█████████ | 1234/1358 [1:35:57<09:10,  4.44s/it]

 91%|█████████ | 1235/1358 [1:36:07<12:06,  5.91s/it]

 91%|█████████ | 1236/1358 [1:36:10<10:33,  5.19s/it]

 91%|█████████ | 1237/1358 [1:36:15<10:24,  5.16s/it]

 91%|█████████ | 1238/1358 [1:36:20<09:53,  4.94s/it]

 91%|█████████ | 1239/1358 [1:36:23<08:56,  4.51s/it]

 91%|█████████▏| 1240/1358 [1:36:28<08:58,  4.56s/it]

 91%|█████████▏| 1241/1358 [1:36:32<08:38,  4.43s/it]

 91%|█████████▏| 1242/1358 [1:36:40<10:51,  5.62s/it]

 92%|█████████▏| 1243/1358 [1:36:43<09:00,  4.70s/it]

 92%|█████████▏| 1244/1358 [1:36:50<10:26,  5.50s/it]

 92%|█████████▏| 1245/1358 [1:36:56<10:13,  5.43s/it]

 92%|█████████▏| 1246/1358 [1:37:02<10:45,  5.76s/it]

 92%|█████████▏| 1247/1358 [1:37:05<08:44,  4.73s/it]

 92%|█████████▏| 1248/1358 [1:37:11<09:37,  5.25s/it]

 92%|█████

In [35]:
df_systran_en_ru_fr.head(100)

,word,local_word,examples,local_examples
0,0,word,"[с, л, о, в, о]","[с, л, о, в, о]"
1,1,and,[и],[и]
2,2,and,[а],[а]
3,3,and,[и],[и]
4,4,and,"[н, о]","[н, о]"
5,5,and,[а],[а]
6,6,that,"[т, а, к]","[т, а, к]"
7,7,that,"[т, о]","[no_translation, о]"
8,8,that,"[к, о, т, о, р, ы, й, , /, , к, т, о, , /, ...","[к, о, т, о, р, ы, й, , /, , к, т, о, , /, ..."
9,9,that,"[т, о, т]","[т, о, т]"
